#eLDW
Description: This script combines and reformats elicited linguistic data. See the [GitHub page](https://github.com/rgriscom/Elicited-Linguistic-Data-Wrangler) for more information.

**How to use the script**
1. Choose the options in the first form and press the first play button.
2. Fill out the information in the generated form and press the second play button.
3. Press the "Browse" button to upload your files.


In [ ]:
import uuid, ipywidgets as widgets, pandas as pd
from IPython.display import display
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual

#@title Settings
Operating_System = 'Linux' #@param ["Linux", "MacOS", "Windows"]
Text_Data_Source = '.csv' #@param [".csv", ".TextGrid", ".eaf", ".flextext", "None"]
Timecode_Data_Source = '.TextGrid' #@param [".csv", ".TextGrid", ".eaf", ".flextext","None"]
Add_REFID_Tier = True #@param {type:"boolean"}

Sounding_Interval_Text = ''
CSV_Timecode_Format = 'Seconds'
Expand_Time_Segments = False
Amount_to_Expand = 10

### Text data settings
if Text_Data_Source == '.flextext':
  column_names = []
  column_types = []
  column_lgs = []
if Text_Data_Source == '.csv' or Text_Data_Source == '.eaf' or Text_Data_Source == '.TextGrid':
  column_names = []
  column_types = []
  column_lgs = []
  number_of_columns = 3
  column_visibility = [1,1,1,0,0,0,0,0,0,0]
  out = widgets.Output()
  display(out)
  CSV_Delimiter = "Comma"
  column_name_entry_list = []
  column_type_entry_list = []
  column_lg_entry_list = []
  First_Row_Is_Column_Names = False
  j = 0
  @out.capture()
  def name_handle_submit(change):
    try:
      column_names[int(change['owner'].description.split(' name:')[0])-1] = change['new']
    except:
      column_names.append(change['new'])

  @out.capture()
  def type_handle_submit(change):
    try:
      column_types[int(change['owner'].description.split(' type:')[0])-1] = change['new']
    except:
      column_types.append(change['new'])

  @out.capture()
  def lg_handle_submit(change):
    try:
      column_lgs[int(change['owner'].description.split(' ISO 639:')[0])-1] = change['new']
    except:
      column_lgs.append(change['new'])

  @out.capture()
  def slider_handle_submit(change):  
    global number_of_columns
    global First_Row_Is_Column_Names 
    number_of_columns = change['new']
    for i in range(10):
      if (i+1) <= number_of_columns:
        if column_type_entry_list[i].layout.display == 'none':
          column_type_entry_list[i].layout.display = 'flex'
        if column_lg_entry_list[i].layout.display == 'none':
          column_lg_entry_list[i].layout.display = 'flex'
        if column_name_entry_list[i].layout.display == 'none' and First_Row_Is_Column_Names == False:
          column_name_entry_list[i].layout.display = 'flex'
      if (i+1) > number_of_columns:
        if column_type_entry_list[i].layout.display == 'flex':
          column_type_entry_list[i].layout.display = 'none'
        if column_lg_entry_list[i].layout.display == 'flex':
          column_lg_entry_list[i].layout.display = 'none'
        if column_name_entry_list[i].layout.display == 'flex':
          column_name_entry_list[i].layout.display = 'none'
  

  column_slider = widgets.IntSlider(
      value=3,
      min=1,
      max=10,
      step=1,
      description='Column_slider',
      disabled=False,
      continuous_update=True,
      orientation='horizontal',
      readout=True,
      readout_format='d',
      style=dict(description_width='initial')
  )
  display(column_slider)  
  
  for i in range(10):
    if First_Row_Is_Column_Names == False:
      column_name_entry_list.append(widgets.Text(
        value='',
        placeholder='Name',
        description=str(i+1) + " name:",
        continuous_update=True,
        disabled=False,
        style=dict(description_width='initial')))
      display(column_name_entry_list[i])
      if (i+1) > number_of_columns:
        column_name_entry_list[i].layout.display = 'none'
      else:
        column_name_entry_list[i].layout.display = 'flex'
    column_type_entry_list.append(widgets.Dropdown(
      options=['Select One','Transcription', 'Translation', 'Notes'],
      value='Select One',
      description=str(i+1) + " type:",
      disabled=False,
      style=dict(description_width='initial')))
    display(column_type_entry_list[i])
    if (i+1) > number_of_columns:
      column_type_entry_list[i].layout.display = 'none'
    else:
      column_type_entry_list[i].layout.display = 'flex'
    column_lg_entry_list.append(widgets.Text(
      value='',
      placeholder='ISO 639 language code',
      description=str(i+1) + " ISO 639:",
      continuous_update=True,
      disabled=False,
      style=dict(description_width='initial')))
    display(column_lg_entry_list[i])
    if (i+1) > number_of_columns:
      column_lg_entry_list[i].layout.display = 'none'
    else:
      column_lg_entry_list[i].layout.display = 'flex'

  column_slider.observe(slider_handle_submit, names='value')

  for i in range(number_of_columns):
    if First_Row_Is_Column_Names == False:
      column_name_entry_list[i].observe(name_handle_submit, names='value')
    column_type_entry_list[i].observe(type_handle_submit, names='value')
    column_lg_entry_list[i].observe(lg_handle_submit, names='value')

if Text_Data_Source == '.csv':
  @out.capture()
  def delimiter_handle_submit(change):
    global CSV_Delimiter
    CSV_Delimiter = change['new']
  
  @out.capture()
  def first_row_handle_submit(change):
    global First_Row_Is_Column_Names
    First_Row_Is_Column_Names = change['new']
    for i in range(10):
      if First_Row_Is_Column_Names == True:
          column_name_entry_list[i].layout.display = 'none'
      if First_Row_Is_Column_Names == False and (i+1) <= number_of_columns:
          column_name_entry_list[i].layout.display = 'flex'
  csv_delimiter_dropdown = widgets.Dropdown(
      options=['Comma','Tab','Semi-colon'],
      value='Comma',
      description="CSV Delimiter:",
      disabled=False,
      style=dict(description_width='initial'),
      layout=dict(width='400px')
      )
  display(csv_delimiter_dropdown)
  First_Row_Is_Column_Names_checkbox = widgets.Checkbox(
    value=False,
    description='First row of CSV is column titles?',
    disabled=False,
    indent=False,
    style=dict(description_width='initial'),
    layout=dict(width='400px')
  )
  display(First_Row_Is_Column_Names_checkbox)
  csv_delimiter_dropdown.observe(delimiter_handle_submit, names='value')
  First_Row_Is_Column_Names_checkbox.observe(first_row_handle_submit, names='value')
  
###Time code settings
if Timecode_Data_Source != 'None':
  @out.capture()
  def time_segments_checkbox_handle_submit(change):
    global Expand_Time_Segments
    Expand_Time_Segments = change['new']
    if Expand_Time_Segments == True:
      Amount_to_Expand_slider.layout.display = 'flex'
    if Expand_Time_Segments == False:
      Amount_to_Expand_slider.layout.display = 'none'

  @out.capture()
  def amount_to_expand_slider_handle_submit(change):
    global Amount_to_Expand
    Amount_to_Expand = change['new']

  Expand_Time_Segments_checkbox = widgets.Checkbox(
    value=False,
    description='Expand duration of time segments',
    disabled=False,
    indent=False,
    style=dict(description_width='initial')
  )
  display(Expand_Time_Segments_checkbox)
  Amount_to_Expand_slider = widgets.IntSlider(
      value=10,
      min=0,
      max=100,
      step=10,
      description='Amount',
      disabled=False,
      continuous_update=True,
      orientation='horizontal',
      readout=True,
      readout_format='d',
  )
  display(Amount_to_Expand_slider)
  Amount_to_Expand_slider.layout.display = 'none'
  Amount_to_Expand_slider.observe(amount_to_expand_slider_handle_submit, names='value')
  Expand_Time_Segments_checkbox.observe(time_segments_checkbox_handle_submit, names='value')

if Timecode_Data_Source == '.TextGrid':
  @out.capture()
  def sound_interval_submit(change):
    global Sounding_Interval_Text
    Sounding_Interval_Text = change['new']

  Sounding_Interval_textbox = widgets.Text(
      value='',
      placeholder='Sounding Interval in TextGrid file',
      description="Sounding Interval",
      continuous_update=True,
      disabled=False,
      style=dict(description_width='initial'),
      layout = widgets.Layout(width='50%')
  )
  display(Sounding_Interval_textbox)

  Sounding_Interval_textbox.observe(sound_interval_submit, names='value')

if Timecode_Data_Source == '.csv':
  @out.capture()
  def csv_time_submit(change):
    global CSV_Timecode_Format
    CSV_Timecode_Format = change['new']

  CSV_Timecode_Format_dropdown = widgets.Dropdown(
      options=['Seconds','Milliseconds'],
      value='Seconds',
      description="CSV Time Format: ",
      disabled=False,
      style=dict(description_width='initial')
  )
  display(CSV_Timecode_Format_dropdown)

  CSV_Timecode_Format_dropdown.observe(csv_time_submit, names='value')


   
  


In [ ]:
#@title
try:
  !rm -rf Output
  !mkdir Output
except:
  !mkdir Output
from google.colab import files
import datetime
now = datetime.datetime.now()
uploaded = files.upload()     
wrong_file_format = 0

if First_Row_Is_Column_Names == True or Text_Data_Source == ".flextext" or Text_Data_Source == "None":
  column_names = []


list_of_files = []
for fn in uploaded.keys():
  filename_stem = fn.split(".")[0]
  if filename_stem not in list_of_files:
    list_of_files.append(filename_stem)

#For each file in the list of files
for fn in list_of_files:
  new_csv_name = "Output/" + fn + ".csv"
  new_textgrid_name = "Output/" + fn + ".TextGrid"
  new_flextext_name = "Output/" + fn + ".flextext"
  new_eaf_name = "Output/" + fn + ".eaf"
  columns_list = column_names.copy()
  columns_list.extend(['REFID','XMIN','XMAX'])
  tsv_df = pd.DataFrame(columns=columns_list)  

#INPUT PROCESSES
  #If text source is being used
  if Text_Data_Source != None:
    #Determine the exact extension
    if fn + Text_Data_Source in uploaded.keys():
      extension = Text_Data_Source
    if fn + Text_Data_Source.upper() in uploaded.keys():
      extension = Text_Data_Source.upper()
    if fn + Text_Data_Source.lower() in uploaded.keys():
      extension = Text_Data_Source.lower()
    print("Processing text from file: " + fn + extension)
    
    with open(fn + extension) as text_source:
      annotation_counter = 1
      
      #.CSV text extraction process
      if Text_Data_Source == ".csv":
        if CSV_Delimiter == "Comma":
          delim = ","
        if CSV_Delimiter == "Tab":
          delim = "\t"
        if CSV_Delimiter == "Semi-colon":
          delim = ";"
        names_check = False
        column_data = []
        for line in text_source:
          temp = line.split('\n')
          rows = temp[0].split(delim)
          if First_Row_Is_Column_Names == True and names_check == False:
            for i in rows:
              column_names.append(i)
            total_columns = len(column_names)
            print("Column names from first row: ")
            print(column_names)
            names_check = True
          else:
            counter = 1
            while counter < (total_columns + 1):
              try:
                
                tsv_df.at[annotation_counter, column_names[(counter - 1)]] = rows[(counter - 1)]
              except IndexError:
                
                tsv_df.at[annotation_counter, column_names[(counter - 1)]] = ""                                  
              counter += 1
            annotation_counter = annotation_counter + 1
       

      #.TextGrid text extraction process
      if Text_Data_Source == ".TextGrid":
        tier_counter = 0
        annotation_counter = 1
        for line in text_source:
          if "name = \"" in line:
            tier_name = line.split('name = \"')[1].split("\"")[0]
            if tier_name in column_names:
              tier_counter += 1
              annotation_counter = 1
             
          if "text = \"" in line and "text = \"\"" not in line:
            annotation = line.split('text = \"')[1].split("\"")[0]
            tsv_df.at[annotation_counter, tier_name] = annotation
            annotation_counter += 1
         

        
    

      #.flextext text extraction process
      if Text_Data_Source == ".flextext": 
        tier_counter = 0
        annotation_counter = 0
        for line in text_source:
          if 'type=\"txt\"' in line:
            tier_language = line.split('lang=\"')[1].split('\"')[0]
            tier_name = 'A_Transcription-txt-' + tier_language 
            if tier_name not in column_names:
              column_names.append(tier_name)
              column_lgs.append(tier_language)
              column_types.append('Transcription')
              tier_counter += 1
            annotation = line.split('type=\"txt\">')[1].split("</item>")[0]
            annotation_counter += 1
            tsv_df.at[annotation_counter, tier_name] = annotation
          if 'type=\"gls\"' in line: 
            tier_language = line.split('lang=\"')[1].split('\"')[0]
            tier_name = 'A_Translation-gls-' + tier_language           
            if tier_name not in column_names:
              column_names.append(tier_name)
              column_lgs.append(tier_language)
              column_types.append('Translation')
              tier_counter += 1
            annotation = line.split('type=\"gls\">')[1].split("</item>")[0]
            
            tsv_df.at[annotation_counter, tier_name] = annotation
        number_of_columns = len(column_names)
        
            

        


      #.eaf text extraction process
      if Text_Data_Source == ".eaf":
        tier_counter = 0
        annotation_counter = 1
        for line in text_source:
          if "TIER_ID=" in line and "/>" not in line:
            tier_name = line.split('TIER_ID=\"')[1].split("\">")[0]
            if tier_name in column_names:
              tier_counter += 1
              annotation_counter = 1
          if "<ANNOTATION_VALUE>" in line: 
            annotation = line.split('<ANNOTATION_VALUE>')[1].split("</ANNOTATION_VALUE>")[0]
            tsv_df.at[annotation_counter, tier_name] = annotation
            annotation_counter += 1
        

  #If timecode source is being used
  if Timecode_Data_Source != None:
    #Determine the exact extension
    if fn + Timecode_Data_Source in uploaded.keys():
      extension = Timecode_Data_Source
    if fn + Timecode_Data_Source.upper() in uploaded.keys():
      extension = Timecode_Data_Source.upper()
    if fn + Timecode_Data_Source.lower() in uploaded.keys():
      extension = Timecode_Data_Source.lower()
    print("Processing timecode from file: " + fn + extension)

    with open(fn + extension) as time_source:
      annotation_counter = 1

      #.TextGrid timecode extraction process
      if Timecode_Data_Source == ".TextGrid":  
        for line in time_source:
          if "xmin" in line:
            temp = line.split('= ')
            temp_xmin = float(temp[1])
          if "xmax" in line:
            temp = line.split('= ')
            temp_xmax = float(temp[1])
          if "name = \"" in line:
            annotation_counter = 1
          if Sounding_Interval_Text != "":
            if "text = \"" + Sounding_Interval_Text + "\"" in line:
              #Expand timecode data duration
              if Expand_Time_Segments == True:
                temp_xmin = temp_xmin - (Amount_to_Expand/1000)
                temp_xmax = temp_xmax + (Amount_to_Expand/1000)
              try:
                tsv_df.at[annotation_counter, 'XMIN'] = temp_xmin
                tsv_df.at[annotation_counter, 'XMAX'] = temp_xmax
                annotation_counter += 1
              except KeyError:
                print("Too many TextGrid segments...")
                print(str(annotation_counter))
          else:
            if "text = \"" in line and "text = \"\"" not in line:
              #Expand timecode data duration
              
              if Expand_Time_Segments == True:
                temp_xmin = temp_xmin - (Amount_to_Expand/1000)
                temp_xmax = temp_xmax + (Amount_to_Expand/1000)
              try:
                tsv_df.at[annotation_counter, 'XMIN'] = temp_xmin
                tsv_df.at[annotation_counter, 'XMAX'] = temp_xmax
                annotation_counter += 1
              except KeyError:
                print("Too many TextGrid segments...")
                print(str(annotation_counter))

          if "tiers? <exists>" in line:
            if Expand_Time_Segments == True:
              final_xmax = temp_xmax + (Amount_to_Expand/1000)
            else:
              final_xmax = temp_xmax 


      #.csv timecode extraction process
      if Timecode_Data_Source == ".csv":
        #CSV_Timecode_Format = 'Seconds' #@param ["Seconds", "Milliseconds"]
        for line in time_source:
              temp_xmin = float(line.split(",")[0])
              temp_xmax = float(line.split(",")[1])
              if CSV_Timecode_Format == 'Seconds':
                temp_xmin = temp_xmin * 1000
                temp_xmax = temp_xmax * 1000
              if Expand_Time_Segments == True:
                temp_xmin = temp_xmin - (Amount_to_Expand/1000)
                temp_xmax = temp_xmax + (Amount_to_Expand/1000)
              try:
                tsv_df.at[annotation_counter, 'XMIN'] = temp_xmin
                tsv_df.at[annotation_counter, 'XMAX'] = temp_xmax
                annotation_counter += 1
              except KeyError:
                print("Too many timecode segments...")
        if Expand_Time_Segments == True:
          final_xmax = temp_xmax + (Amount_to_Expand/1000)
        else:
          final_xmax = (temp_xmax/1000)
            
      #.eaf timecode extraction process
      if Timecode_Data_Source == ".eaf":
        eaf_toggle = False 
        for line in time_source:
          if "<TIME_SLOT TIME_SLOT_ID=" in line:
            if eaf_toggle == True:
              temp_xmax = float(line.split("TIME_VALUE=\"")[1].split("\"/>")[0])
              eaf_toggle = False
                #Expand timecode data duration
              if Expand_Time_Segments == True:
                temp_xmin = temp_xmin - (Amount_to_Expand/1000)
                temp_xmax = temp_xmax + (Amount_to_Expand/1000)
              try:
                tsv_df.at[annotation_counter, 'XMIN'] = temp_xmin/1000
                tsv_df.at[annotation_counter, 'XMAX'] = temp_xmax/1000
                annotation_counter += 1
              except KeyError:
                print("Too many timecode segments...")
            else:
              temp_xmin = float(line.split("TIME_VALUE=\"")[1].split("\"/>")[0])
              eaf_toggle = True
        if Expand_Time_Segments == True:
          final_xmax = (temp_xmax + (Amount_to_Expand/1000))/1000
        else:
          final_xmax = (temp_xmax/1000)
        
          
      #.flextext timecode extraction process
      if Timecode_Data_Source == ".flextext":
        for line in time_source:
          if "begin-time-offset=" in line:
            temp_xmin = float(line.split("<phrase begin-time-offset=\"")[1].split("\"")[0])
          if "end-time-offset=" in line:    
            temp_xmax = float(line.split("end-time-offset=\"")[1].split("\"")[0])
              #Expand timecode data duration
            if Expand_Time_Segments == True:
              temp_xmin = temp_xmin - (Amount_to_Expand/1000)
              temp_xmax = temp_xmax + (Amount_to_Expand/1000)
            try:
              tsv_df.at[annotation_counter, 'XMIN'] = temp_xmin / 1000
              tsv_df.at[annotation_counter, 'XMAX'] = temp_xmax / 1000
              annotation_counter += 1
            except KeyError:
              print("Too many timecode segments...")
        if Expand_Time_Segments == True:
          final_xmax = temp_xmax + (Amount_to_Expand/1000)
        else:
          final_xmax = temp_xmax / 1000

  #Add the REFID labels to the DataFrame
  total_annotations = annotation_counter - 1
  for i in range(annotation_counter):
    tsv_df.at[i, 'REFID'] = filename_stem + "_" + str(i)
  
  #Timecode expansion check
  if Expand_Time_Segments == True:
    annotation_counter = 2
    expansion_check = False
    while annotation_counter < (total_annotations + 1):
      if tsv_df.at[annotation_counter, 'XMIN'] < tsv_df.at[(annotation_counter - 1), 'XMAX'] and expansion_check == False:
        print("Time expansion error with file " + filename_stem + ". Reverting to original timecode.")
        expansion_check = True
        for i in range(total_annotations):
          if i > 0:
            tsv_df.at[i, 'XMIN'] = tsv_df.at[i, 'XMIN'] + (Amount_to_Expand/1000)
            tsv_df.at[i, 'XMAX'] = tsv_df.at[i, 'XMAX'] - (Amount_to_Expand/1000)
      annotation_counter+=1
    

   
#OUTPUT PROCESSES
  #Populate new CSV file
  with open(new_csv_name, "w") as csv_new:
    
    
    annotation_counter = 1
    while annotation_counter < (total_annotations - 1):
      if Timecode_Data_Source != None:
        csv_new.write(tsv_df.at[annotation_counter, 'REFID'] + ',' + str(tsv_df.at[annotation_counter, 'XMIN']) + ',' + str(tsv_df.at[annotation_counter, 'XMAX']))
      counter = 1
      if Text_Data_Source != None:
        while counter < (total_columns + 1):

          csv_new.write(',' + tsv_df.at[annotation_counter, column_names[(counter - 1)]]) 
          counter += 1
      csv_new.write('\n')
      annotation_counter += 1
    
         
    
    #Populate new .flextext file
    if Text_Data_Source != "None":
      with open(new_flextext_name, "w") as new_flextext:
        document_uuid = str(uuid.uuid4())
        media_uuid = str(uuid.uuid4())
        analysis_lg_code = "en"
        document_title = filename_stem

        new_flextext.write("<?xml version=\"1.0\" encoding=\"UTF-8\"?>\n")
        new_flextext.write("<document version=\"2\">\n")
        new_flextext.write("    <interlinear-text guid=\"" + document_uuid +  "\">\n")
        new_flextext.write("        <item lang=\"" + analysis_lg_code + "\" type=\"title\">" + document_title + "</item>\n")
        new_flextext.write("        <paragraphs>\n")
        

        #Cycle through all of the annotations    
        annotation_counter = 1
        while annotation_counter < (total_annotations + 1):
          paragraph_uuid = str(uuid.uuid4())
          phrase_uuid = str(uuid.uuid4())
          new_flextext.write("            <paragraph guid=\"" + paragraph_uuid + "\">\n")
          new_flextext.write("                <phrases>\n")
          if Timecode_Data_Source != None:
            new_flextext.write("                    <phrase begin-time-offset=\"" + str(int(tsv_df.at[annotation_counter,'XMIN']*1000)) + "\"\n")
            new_flextext.write("                        end-time-offset=\"" + str(int(tsv_df.at[annotation_counter, 'XMAX']*1000)) + "\"\n")
          else:
            new_flextext.write("                    <phrase\n")
          new_flextext.write("                        guid=\"" + phrase_uuid + "\"\n")
          new_flextext.write("                        media-file=\"" + media_uuid + "\" speaker=\"A\">\n")
          for i in range(number_of_columns):
            if column_types[i] == "Transcription":
              new_flextext.write("                        <item lang=\"" + column_lgs[i] + "\" type=\"txt\">" + str(tsv_df.at[annotation_counter, column_names[i]]) + "</item>\n")
          new_flextext.write("                        <words/>\n")
          if "zxx" not in column_lgs:
            new_flextext.write("                        <item lang=\"zxx\" type=\"gls\">" + tsv_df.at[annotation_counter, 'REFID'] + "</item>\n")
          for i in range(number_of_columns):
            if column_types[i] == "Translation":
              new_flextext.write("                        <item lang=\"" + column_lgs[i] + "\" type=\"gls\">" + str(tsv_df.at[annotation_counter, column_names[i]]) + "</item>\n")
          new_flextext.write("                    </phrase>\n")
          new_flextext.write("                </phrases>\n")
          new_flextext.write("            </paragraph>\n")
          annotation_counter += 1

        new_flextext.write("        </paragraphs>\n")
        new_flextext.write("        <media-files offset-type=\"\">\n")
        new_flextext.write("            <media guid=\"" + media_uuid + "\" location=\"\"/>\n")
        new_flextext.write("        </media-files>\n")
        new_flextext.write("    </interlinear-text>\n")
        new_flextext.write("</document>")
      new_flextext.close()      

    #Populate new .TextGrid file, with three tiers: unique REF ID, transcription, and translation    
     
    if Timecode_Data_Source != "None":   
      with open(new_textgrid_name, "w") as textgrid_new:                          
        #Fill out the beginning of the TextGrid file
        textgrid_new.write("File type = \"ooTextFile\"\nObject class = \"TextGrid\"\n\nxmin = 0\nxmax = " + str(final_xmax) + "\ntiers? <exists>\nsize = " + str((total_columns + 1)) + "\nitem []:\n")
        tier_counter = 1
        #Fill out the beginning of each tier in the TextGrid
        while tier_counter < (total_columns + 2):                                
          textgrid_new.write("    item [" + str(tier_counter) + "]:\n")
          textgrid_new.write("        class = \"IntervalTier\"\n")
          if tier_counter == 1:
            textgrid_new.write("        name = \"REFID\"\n")
          else:
            if Text_Data_Source != "None":
              textgrid_new.write("        name = \"" + column_names[(tier_counter - 2)] + "\"\n")               
          textgrid_new.write("        xmin = 0\n")
          textgrid_new.write("        xmax = " + str(final_xmax) + "\n")
          textgrid_new.write("        intervals: size = " + str((total_annotations * 2) + 1) + "\n")
          #Fill out each TextGrid interval using the dictionary
          annotation_counter = 1
          while annotation_counter < (total_annotations + 1):
            if annotation_counter == 1:
              temp_prev_xmax = 0
            else:
              temp_prev_xmax = tsv_df.at[(annotation_counter - 1),'XMAX']
            temp_next_xmin = tsv_df.at[annotation_counter,'XMIN']
            textgrid_new.write("        intervals [" + str(((annotation_counter * 2) - 1)) + "]:\n")
            if annotation_counter == 1:
              textgrid_new.write("            xmin = 0\n")
            else:
              textgrid_new.write("            xmin = " + str(temp_prev_xmax) + "\n")
            textgrid_new.write("            xmax = " + str(temp_next_xmin) + "\n")
            textgrid_new.write("            text = \"\"\n")
            textgrid_new.write("        intervals [" + str(annotation_counter * 2) + "]:\n")
            textgrid_new.write("            xmin = " + str(tsv_df.at[annotation_counter, 'XMIN']) + "\n")
            textgrid_new.write("            xmax = " + str(tsv_df.at[annotation_counter, 'XMAX']) + "\n")
            if tier_counter == 1:
              textgrid_new.write("            text = \"" + str(tsv_df.at[annotation_counter, 'REFID']) + "\"\n")
            else:
              if Text_Data_Source != "None":
                textgrid_new.write("            text = \"" + str(tsv_df.at[annotation_counter, column_names[(tier_counter - 2)]]) + "\"\n")
              else:
                textgrid_new.write("            text = \"\"\n")
            annotation_counter += 1
          textgrid_new.write("        intervals [" + str((((annotation_counter - 1) * 2) + 1)) + "]:\n")
          xmax_of_final_annotation = tsv_df.at[total_annotations, 'XMAX']
          textgrid_new.write("            xmin = " + str(xmax_of_final_annotation) + "\n")
          textgrid_new.write("            xmax = " + str(final_xmax) + "\n")
          textgrid_new.write("            text = \"\"\n")
          tier_counter += 1
      textgrid_new.close()                                 

#Populate new .EAF file
#Fill out the beginning of the EAF file
    if Timecode_Data_Source != "None":
      with open(new_eaf_name, "w") as eaf_new:
        eaf_new.write("<?xml version=\"1.0\" encoding=\"UTF-8\"?>\n")
        eaf_new.write("<ANNOTATION_DOCUMENT AUTHOR=\"unspecified\" DATE=\"" + str(now.year) + "-" + str(now.month) + "-" + str(now.day) + "T" + str(now.hour) + ":" + str(now.minute) + ":" + str(now.second) + "-08:00\" FORMAT=\"3.0\" VERSION=\"3.0\" xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xsi:noNamespaceSchemaLocation=\"http://www.mpi.nl/tools/elan/EAFv3.0.xsd\">\n")
        eaf_new.write("    <HEADER MEDIA_FILE=\"\" TIME_UNITS=\"milliseconds\">\n")
  #Fixes slashes in Windows directories
        if Operating_System == 'Windows':                   
          eaf_new.write("        <MEDIA_DESCRIPTOR MEDIA_URL=\"file:///" + filename_stem + ".wav\" MIME_TYPE=\"audio/x-wav\" RELATIVE_MEDIA_URL=\"./" + filename_stem + ".wav\"/>\n")
        else:
          eaf_new.write("        <MEDIA_DESCRIPTOR MEDIA_URL=\"file://" + filename_stem + ".wav\" MIME_TYPE=\"audio/x-wav\" RELATIVE_MEDIA_URL=\"./" + filename_stem + ".wav\"/>\n")
        eaf_new.write("        <PROPERTY NAME=\"URN\">urn:nl-mpi-tools-elan-eaf:93cd58ea-4af9-44d5-a6d5-d468217ccf5e</PROPERTY>\n")
        eaf_new.write("        <PROPERTY NAME=\"lastUsedAnnotationId\">" + str((5 * total_annotations)) + "</PROPERTY>\n")
        eaf_new.write("    </HEADER>\n")
        eaf_new.write("    <TIME_ORDER>\n")                       
  #Fill out time slots
        annotation_counter = 1
        while annotation_counter < (total_annotations + 1):
          eaf_new.write("        <TIME_SLOT TIME_SLOT_ID=\"ts" + str((annotation_counter * 2) - 1) + "\" TIME_VALUE=\"" + str(int(1000 * tsv_df.at[annotation_counter, 'XMIN'])) + "\"/>\n")
          eaf_new.write("        <TIME_SLOT TIME_SLOT_ID=\"ts" + str((annotation_counter * 2)) + "\" TIME_VALUE=\"" + str(int(1000 * tsv_df.at[annotation_counter, 'XMAX'])) + "\"/>\n")
          annotation_counter += 1
  #Fill out RFID annotations
        eaf_new.write("    </TIME_ORDER>\n")
        eaf_new.write("    <TIER DEFAULT_LOCALE=\"en\" LINGUISTIC_TYPE_REF=\"REFID\" TIER_ID=\"REFID\">\n")
        annotation_counter = 1
        while annotation_counter < (total_annotations + 1):
          eaf_new.write("        <ANNOTATION>\n")
          eaf_new.write("            <ALIGNABLE_ANNOTATION ANNOTATION_ID=\"a" + str((2 * total_annotations) + annotation_counter) + "\" TIME_SLOT_REF1=\"ts" + str((annotation_counter * 2) -1) + "\" TIME_SLOT_REF2=\"ts" + str(annotation_counter * 2) + "\">\n")
          eaf_new.write("                <ANNOTATION_VALUE>" + str(tsv_df.at[annotation_counter, 'REFID']) + "</ANNOTATION_VALUE>\n")
          eaf_new.write("            </ALIGNABLE_ANNOTATION>\n")
          eaf_new.write("        </ANNOTATION>\n")
          annotation_counter += 1
        eaf_new.write("    </TIER>\n")                
  #Fill out transcription and translation annotations
        if Text_Data_Source != "None":
          tier_counter = 1
          while tier_counter < total_columns + 1:                        
            eaf_new.write("    <TIER DEFAULT_LOCALE=\"en\" LINGUISTIC_TYPE_REF=\"" + column_names[(tier_counter - 1)] + "\" PARENT_REF=\"REFID\" TIER_ID=\"" + column_names[(tier_counter - 1)] + "\">\n")
            annotation_counter = 1
            while annotation_counter < (total_annotations + 1):
              eaf_new.write("        <ANNOTATION>\n")
              eaf_new.write("            <REF_ANNOTATION ANNOTATION_ID=\"a" + str(((tier_counter + 2) * total_annotations) + annotation_counter) + "\" ANNOTATION_REF=\"a" + str((2 * total_annotations) + annotation_counter) + "\">\n")
              eaf_new.write("                <ANNOTATION_VALUE>" + str(tsv_df.at[annotation_counter, column_names[(tier_counter - 1)]]) + "</ANNOTATION_VALUE>\n")
              eaf_new.write("            </REF_ANNOTATION>\n")
              eaf_new.write("        </ANNOTATION>\n")
              annotation_counter += 1
            eaf_new.write("    </TIER>\n")
            tier_counter += 1
                      
                                
  #Fill out end of the EAF file
        eaf_new.write("    <LINGUISTIC_TYPE GRAPHIC_REFERENCES=\"false\" LINGUISTIC_TYPE_ID=\"REFID\" TIME_ALIGNABLE=\"true\"/>\n")
        if Text_Data_Source != "None":
          counter = 1
          while counter < (total_columns + 1):
            eaf_new.write("    <LINGUISTIC_TYPE CONSTRAINTS=\"Symbolic_Association\" GRAPHIC_REFERENCES=\"false\" LINGUISTIC_TYPE_ID=\"" + column_names[(counter - 1)] + "\" TIME_ALIGNABLE=\"false\"/>\n")    
            counter += 1
        eaf_new.write("    <LOCALE COUNTRY_CODE=\"US\" LANGUAGE_CODE=\"en\"/>\n")
        eaf_new.write("    <CONSTRAINT DESCRIPTION=\"Time subdivision of parent annotation's time interval, no time gaps allowed within this interval\" STEREOTYPE=\"Time_Subdivision\"/>\n")
        eaf_new.write("    <CONSTRAINT DESCRIPTION=\"Symbolic subdivision of a parent annotation. Annotations refering to the same parent are ordered\" STEREOTYPE=\"Symbolic_Subdivision\"/>\n")
        eaf_new.write("    <CONSTRAINT DESCRIPTION=\"1-1 association with a parent annotation\" STEREOTYPE=\"Symbolic_Association\"/>\n")
        eaf_new.write("    <CONSTRAINT DESCRIPTION=\"Time alignable annotations within the parent annotation's time interval, gaps are allowed\" STEREOTYPE=\"Included_In\"/>\n")
        eaf_new.write("</ANNOTATION_DOCUMENT>")
      
    print("Processing complete for: " + fn)
zip_filename = "eLDW_Output_" + now.strftime("%Y%m%d") + ".zip"
!zip -r $zip_filename Output/
files.download(zip_filename)